# Logistic Regressor for ChiP-Seq Dataset

Written by Micah Lessnick

Logistic regressor to ID True and False peaks in EWS/FLI ChiP-Seq datasets. Trained across 4 cell lines provided by Nationwide Children's Hospital cancer research center.

Mount program to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression

Load CSV file

In [ ]:
whole_df = pd.read_csv('/content/gdrive/MyDrive/Ford_Research_Models/peakDataAllNC.csv')
whole_df

#dfs containing remaining cell lines (for additional testing)
whole_ews_df = pd.read_csv('/content/gdrive/MyDrive/Ford_Research_Models/peakDataEws.csv')

whole_sknmc_df = pd.read_csv('/content/gdrive/MyDrive/Ford_Research_Models/peakDataSknmc.csv')

whole_tc71_df = pd.read_csv('/content/gdrive/MyDrive/Ford_Research_Models/peakDataTc71.csv')

whole_ews_df

,width,log2FC_score,log2FC,X.log10pval,X.log10FDR,class
0,501,183.719443,6.123231,15.224318,13.232433,1
1,501,5.376616,3.091402,4.187096,3.478807,1
2,501,-28.353692,2.517987,3.042289,2.561334,1
3,501,35.721267,3.607262,5.728388,4.761781,1
4,501,11.215371,3.190661,4.533287,3.761623,1
...,...,...,...,...,...,...
23112,501,-27.348917,2.535068,2.406887,2.063259,0
23113,501,-33.543010,2.429769,1.944525,1.693271,0
23114,501,95.413111,4.622023,3.148077,2.645609,0
23115,501,10.852415,3.184491,3.140340,2.639374,0


Separate labels from data

In [ ]:
full_target_df = whole_df.loc[:, 'class']
full_target_df #df holding only the labels in order as they appear from the original data

ews_target_df = whole_ews_df.loc[:, 'class']

sknmc_target_df = whole_sknmc_df.loc[:, 'class']

tc71_target_df = whole_tc71_df.loc[:, 'class']

full_data_df = whole_df.drop('class', axis=1)
#full_data_df.drop('log2FC_score', axis=1, inplace=True)#remove duplicate distributed features (allows
#full_data_df.drop('X.log10pval', axis=1, inplace=True)  #normalization to occur correctly)
full_data_df #df holding everything but the labels

ews_data_df = whole_ews_df.drop('class', axis=1) #r script already removed the seqnames column from these dfs
#ews_data_df.drop('log2FC_score', axis=1, inplace=True)
#ews_data_df.drop('X.log10pval', axis=1, inplace=True)

sknmc_data_df = whole_sknmc_df.drop('class', axis=1)
#sknmc_data_df.drop('log2FC_score', axis=1, inplace=True)
#sknmc_data_df.drop('X.log10pval', axis=1, inplace=True)

tc71_data_df = whole_tc71_df.drop('class', axis=1)
#tc71_data_df.drop('log2FC_score', axis=1, inplace=True)
#tc71_data_df.drop('X.log10pval', axis=1, inplace=True)

Prepare test, training, and validation sets

In [ ]:
#split training and test data
train_data, test_data, train_target, test_target = train_test_split(
    full_data_df, full_target_df, train_size=0.8, random_state=17
)
#split full training into training and validation sets (may not be needed for logistic regressor)
#train_data, valid_data, train_target, valid_target = train_test_split(
#    train_full_data, train_full_target, train_size=0.8, random_state=17
#)

Reset random number seeds (ensures repeatable results)

In [ ]:
np.random.seed(17)
tf.random.set_seed(17)

Capture mean and standard deviation of training set for use in normalization function

In [ ]:
#compute mean and std for training dataset, will be used in zscore function
data_mean = np.mean(train_data)
data_std = np.std(train_data)

Normalize dataset based on zscore of train_data

In [ ]:
#function to compute zscore of any dataset provided, based on train_data zscore
def zscoreNormRelative(data):
  normData = (data-data_mean)/data_std
  return (normData)

train_data = zscoreNormRelative(train_data)
test_data = zscoreNormRelative(test_data)

#function to compute zscore of provided df (guarantees returned df will have mean = 0, std = 1)
def zscoreNormActual(dataFrame):
  norm_mean = np.mean(dataFrame)
  norm_std = np.std(dataFrame)
  return ((dataFrame-norm_mean)/norm_std)

ews_data_df = zscoreNormActual(ews_data_df)
sknmc_data_df = zscoreNormActual(sknmc_data_df)
tc71_data_df = zscoreNormActual(tc71_data_df)

np.mean(sknmc_data_df)

width          -9.978676e-15
log2FC_score    2.181318e-15
log2FC          5.385636e-16
X.log10pval     4.063586e-15
X.log10FDR     -1.878514e-15
dtype: float64

In [ ]:
train_data

,width,log2FC_score,log2FC,X.log10pval,X.log10FDR
282,-0.170796,0.313998,0.313998,0.008237,-0.000515
27407,-0.170796,-0.696208,-0.696208,-0.812107,-0.798331
63243,-0.163728,0.845751,0.845751,-0.350768,-0.354063
24362,-0.170796,-0.926490,-0.926490,-0.890775,-0.870257
33450,-0.180264,-0.254399,-0.254399,0.133869,0.111766
...,...,...,...,...,...
25631,-0.170796,0.294627,0.294627,0.575189,0.566976
42297,-0.180264,-1.369751,-1.369751,-1.086991,-1.046680
34959,-0.180264,-1.196624,-1.196624,-0.710917,-0.708382
64753,-0.163728,0.334153,0.334153,-0.201278,-0.205352


Build/run regressor

In [ ]:
log_reg = LogisticRegression(solver="lbfgs", random_state=17)
log_reg.fit(train_data, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Analyze results

In [ ]:
pred = log_reg.predict(test_data)
conf = sklearn.metrics.confusion_matrix(test_target, pred)
print("ChiP-Seq Regressor:")
print(conf)

print("\nClassification report:")
print(sklearn.metrics.classification_report(test_target, pred, target_names=("false positives", "true peaks")))

print("\nF1 score of the model:")
print(sklearn.metrics.f1_score(test_target, pred))
#f1 score appears to be taking f1 score of true peaks only
print("================================================================================")
#EWS eval
print("\nEvaluate model performance on EWS cell line")

ews_pred = log_reg.predict(ews_data_df)

print("\nF1 score of the model on EWS:")
currentF1Ews = sklearn.metrics.f1_score(ews_target_df, ews_pred)
print(str(currentF1Ews) + "\n")

print("Confusion matrix of current model on EWS:")
print(sklearn.metrics.confusion_matrix(ews_target_df, ews_pred))

#SKNMC eval
print("\nEvaluate model performance on SKNMC cell line")

sknmc_pred = log_reg.predict(sknmc_data_df)

print("\nF1 score of the model on SKNMC:")
currentF1Sknmc = sklearn.metrics.f1_score(sknmc_target_df, sknmc_pred)
print(str(currentF1Sknmc) + "\n")

print("Confusion matrix of current model on SKNMC:")
print(sklearn.metrics.confusion_matrix(sknmc_target_df, sknmc_pred))

#TC71 eval
print("\nEvaluate model performance on TC71 cell line")

tc71_pred = log_reg.predict(tc71_data_df)

print("\nF1 score of the model on TC71:")
currentF1Tc71 = sklearn.metrics.f1_score(tc71_target_df, tc71_pred)
print(str(currentF1Tc71) + "\n")

print("Confusion matrix of current model on TC71:")
print(sklearn.metrics.confusion_matrix(tc71_target_df, tc71_pred))

ChiP-Seq Regressor:
[[10335   863]
 [ 2520  3829]]

Classification report:
                 precision    recall  f1-score   support

false positives       0.80      0.92      0.86     11198
     true peaks       0.82      0.60      0.69      6349

       accuracy                           0.81     17547
      macro avg       0.81      0.76      0.78     17547
   weighted avg       0.81      0.81      0.80     17547


F1 score of the model:
0.6935965945113668

Evaluate model performance on EWS cell line

F1 score of the model on EWS:
0.6239419588875452

Confusion matrix of current model on EWS:
[[13444  1707]
 [ 3580  4386]]

Evaluate model performance on SKNMC cell line

F1 score of the model on SKNMC:
0.7343097821962511

Confusion matrix of current model on SKNMC:
[[9883  371]
 [3130 4838]]

Evaluate model performance on TC71 cell line

F1 score of the model on TC71:
0.5979398299542185

Confusion matrix of current model on TC71:
[[6479  614]
 [4304 3657]]
